Ispired by this notebook.

https://www.kaggle.com/mrkmakr/covid-ae-pretrain-gnn-attn-cnn

Pytorch Implementation.

Model: Convolution + Transfomer + GRU.

Training: Bert-Like Pretraining. Then Fine Tuning.

In [1]:
import numpy as np
import pandas as pd
import ast
from collections import OrderedDict
from fastprogress import progress_bar
from pathlib import Path
from torch import nn
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math

import functools
import itertools
import os
import random
import shutil
import torch
import torch.nn.functional as F

from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, StratifiedKFold, GroupKFold
from torch import nn
from torch.utils.data import Dataset

import functools
from IPython.core.debugger import set_trace


target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
input_cols = ['sequence', 'structure', 'predicted_loop_type']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_Mg_50C', 'deg_error_pH10', 'deg_error_50C']

token_dicts = {
    "sequence": {x: i for i, x in enumerate("ACGU")},
    "structure": {x: i for i, x in enumerate('().')},
    "predicted_loop_type": {x: i for i, x in enumerate("BEHIMSX")}
}


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

### Loader


In [2]:
BASE_PATH = "../OpenVaccine"
MODEL_SAVE_PATH = "../OpenVaccine/pretrains"


def preprocess_inputs(df, cols):
    return np.concatenate([preprocess_feature_col(df, col) for col in cols], axis=2)


def preprocess_feature_col(df, col):
    dic = token_dicts[col]
    dic_len = len(dic)
    seq_length = len(df[col][0])
    ident = np.identity(dic_len)
    # convert to one hot
    arr = np.array(
        df[[col]].applymap(lambda seq: [ident[dic[x]] for x in seq]).values.tolist()
    ).squeeze(1)
    # shape: data_size x seq_length x dic_length
    assert arr.shape == (len(df), seq_length, dic_len)
    return arr


def preprocess(base_data, is_test=False):
    inputs = preprocess_inputs(base_data, input_cols)
    if is_test:
        labels = None
    else:
        labels = np.array(base_data[target_cols].values.tolist()).transpose((0, 2, 1))
        assert labels.shape[2] == len(target_cols)
    assert inputs.shape[2] == 14
    return inputs, labels


def get_bpp_feature(bpp):
    bpp_nb_mean = 0.077522  # mean of bpps_nb across all training data
    bpp_nb_std = 0.08914  # std of bpps_nb across all training data
    bpp_max = bpp.max(-1)[0]
    bpp_sum = bpp.sum(-1)
    bpp_nb = torch.true_divide((bpp > 0).sum(dim=1), bpp.shape[1])
    bpp_nb = torch.true_divide(bpp_nb - bpp_nb_mean, bpp_nb_std)
    return [bpp_max.unsqueeze(2), bpp_sum.unsqueeze(2), bpp_nb.unsqueeze(2)]


@functools.lru_cache(5000)
def load_from_id(id_):
    path = Path(BASE_PATH) / f"bpps/{id_}.npy"
    data = np.load(str(path))
    return data


def get_distance_matrix(leng):
    idx = np.arange(leng)
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1 / Ds
    Ds = Ds[None, :, :]
    Ds = np.repeat(Ds, 1, axis=0)

    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis=3)
    print(Ds.shape)
    return Ds


def get_structure_adj(df):
    Ss = []
    for i in range(len(df)):
        seq_length = df["seq_length"].iloc[i]
        structure = df["structure"].iloc[i]
        sequence = df["sequence"].iloc[i]

        cue = []
        a_structures = OrderedDict([
            (("A", "U"), np.zeros([seq_length, seq_length])),
            (("C", "G"), np.zeros([seq_length, seq_length])),
            (("U", "G"), np.zeros([seq_length, seq_length])),
            (("U", "A"), np.zeros([seq_length, seq_length])),
            (("G", "C"), np.zeros([seq_length, seq_length])),
            (("G", "U"), np.zeros([seq_length, seq_length])),
        ])
        for j in range(seq_length):
            if structure[j] == "(":
                cue.append(j)
            elif structure[j] == ")":
                start = cue.pop()
                a_structures[(sequence[start], sequence[j])][start, j] = 1
                a_structures[(sequence[j], sequence[start])][j, start] = 1

        a_strc = np.stack([a for a in a_structures.values()], axis=2)
        a_strc = np.sum(a_strc, axis=2, keepdims=True)
        Ss.append(a_strc)

    Ss = np.array(Ss)
    return Ss


def create_loader(df, batch_size=1, is_test=False):
    features, labels = preprocess(df, is_test)
    features_tensor = torch.from_numpy(features)
    if labels is not None:
        labels_tensor = torch.from_numpy(labels)
        dataset = VacDataset(features_tensor, df, labels_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, drop_last=False)
    else:
        dataset = VacDataset(features_tensor, df, None)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=False, drop_last=False)
    return loader


class VacDataset(Dataset):
    def __init__(self, features, df, labels=None):
        self.features = features
        self.labels = labels
        self.test = labels is None
        self.ids = df["id"]
        self.score = None
        self.structure_adj = get_structure_adj(df)
        self.distance_matrix = get_distance_matrix(self.structure_adj.shape[1])
        if "score" in df.columns:
            self.score = df["score"]
        else:
            df["score"] = 1.0
            self.score = df["score"]
        self.signal_to_noise = None
        if not self.test:
            self.signal_to_noise = df["signal_to_noise"]
            assert self.features.shape[0] == self.labels.shape[0]
        else:
            assert self.ids is not None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        bpp = torch.from_numpy(load_from_id(self.ids[index]).copy()).float()
        adj = self.structure_adj[index]
        distance = self.distance_matrix[0]
        bpp = np.concatenate([bpp[:, :, None], adj, distance], axis=2)
        if self.test:
            return dict(sequence=self.features[index].float(), bpp=bpp, ids=self.ids[index])
        else:
            return dict(sequence=self.features[index].float(), bpp=bpp,
                        label=self.labels[index], ids=self.ids[index],
                        signal_to_noise=self.signal_to_noise[index],
                        score=self.score[index])


### Model

In [3]:
class Conv1dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv1dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv1d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class Conv2dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv2dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class SeqEncoder(nn.Module):
    def __init__(self, in_dim: int):
        super(SeqEncoder, self).__init__()
        self.conv0 = Conv1dStack(in_dim, 128, 3, padding=1)
        self.conv1 = Conv1dStack(128, 64, 6, padding=5, dilation=2)
        self.conv2 = Conv1dStack(64, 32, 15, padding=7, dilation=1)
        self.conv3 = Conv1dStack(32, 32, 30, padding=29, dilation=2)

    def forward(self, x):
        x1 = self.conv0(x)
        x2 = self.conv1(x1)
        x3 = self.conv2(x2)
        x4 = self.conv3(x3)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        # x = x.permute(0, 2, 1).contiguous()
        # BATCH x 256 x seq_length
        return x


class BppAttn(nn.Module):
    def __init__(self, in_channel: int, out_channel: int):
        super(BppAttn, self).__init__()
        self.conv0 = Conv1dStack(in_channel, out_channel, 3, padding=1)
        self.bpp_conv = Conv2dStack(5, out_channel)

    def forward(self, x, bpp):
        x = self.conv0(x)
        bpp = self.bpp_conv(bpp)
        # BATCH x C x SEQ x SEQ
        # BATCH x C x SEQ
        x = torch.matmul(bpp, x.unsqueeze(-1))
        return x.squeeze(-1)


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerWrapper(nn.Module):
    def __init__(self, dmodel=256, nhead=8, num_layers=2):
        super(TransformerWrapper, self).__init__()
        self.pos_encoder = PositionalEncoding(256)
        encoder_layer = TransformerEncoderLayer(d_model=dmodel, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers)
        self.pos_emb = PositionalEncoding(dmodel)

    def flatten_parameters(self):
        pass

    def forward(self, x):
        x = x.permute((1, 0, 2)).contiguous()
        x = self.pos_emb(x)
        x = self.transformer_encoder(x)
        x = x.permute((1, 0, 2)).contiguous()
        return x, None


class RnnLayers(nn.Module):
    def __init__(self, dmodel, dropout=0.3, transformer_layers: int = 2):
        super(RnnLayers, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.rnn0 = TransformerWrapper(dmodel, nhead=8, num_layers=transformer_layers)
        self.rnn1 = nn.LSTM(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)
        self.rnn2 = nn.GRU(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)

    def forward(self, x):
        self.rnn0.flatten_parameters()
        x, _ = self.rnn0(x)
        if self.rnn1 is not None:
            self.rnn1.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn1(x)
        if self.rnn2 is not None:
            self.rnn2.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn2(x)
        return x

    
class BaseAttnModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(BaseAttnModel, self).__init__()
        self.linear0 = nn.Linear(14 + 3, 1)
        self.seq_encoder_x = SeqEncoder(18)
        self.attn = BppAttn(256, 128)
        self.seq_encoder_bpp = SeqEncoder(128)
        self.seq = RnnLayers(256 * 2, dropout=0.3,
                             transformer_layers=transformer_layers)

    def forward(self, x, bpp):
        bpp_features = get_bpp_feature(bpp[:, :, :, 0].float())
        x = torch.cat([x] + bpp_features, dim=-1)
        learned = self.linear0(x)
        x = torch.cat([x, learned], dim=-1)
        x = x.permute(0, 2, 1).contiguous().float()
        # BATCH x 18 x seq_len
        bpp = bpp.permute([0, 3, 1, 2]).contiguous().float()
        # BATCH x 5 x seq_len x seq_len
        x = self.seq_encoder_x(x)
        # BATCH x 256 x seq_len
        bpp = self.attn(x, bpp)
        bpp = self.seq_encoder_bpp(bpp)
        # BATCH x 256 x seq_len
        x = x.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        bpp = bpp.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        x = torch.cat([x, bpp], dim=2)
        # BATCH x seq_len x 512
        x = self.seq(x)
        return x


class AEModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(AEModel, self).__init__()
        self.seq = BaseAttnModel(transformer_layers=transformer_layers)
        self.linear = nn.Sequential(
            nn.Linear(256 * 2, 14),
            nn.Sigmoid(),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = F.dropout(x, p=0.3)
        x = self.linear(x)
        return x


class FromAeModel(nn.Module):
    def __init__(self, seq, pred_len=68, dmodel: int = 256):
        super(FromAeModel, self).__init__()
        self.seq = seq
        self.pred_len = pred_len
        self.linear = nn.Sequential(
            nn.Linear(dmodel * 2, len(target_cols)),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = self.linear(x)
        x = x[:, :self.pred_len]
        return x

In [4]:
aug_df = pd.read_csv('../OpenVaccine/aug_data1.csv')

def aug_data(df):
    target_df = df.copy()
    new_df = aug_df[aug_df['id'].isin(target_df['id'])]
                         
    del target_df['structure']
    del target_df['predicted_loop_type']
    new_df = new_df.merge(target_df, on=['id','sequence'], how='left')

    df['cnt'] = df['id'].map(new_df[['id','cnt']].set_index('id').to_dict()['cnt'])
    df['log_gamma'] = 100
    df['score'] = 1.0
    df = df.append(new_df[df.columns])
    return df

In [5]:
pseudo_df = pd.read_csv('../OpenVaccine/pseudo_test.csv')

pseudo_df['SN_filter'] = 1
pseudo_df['signal_to_noise'] = 1.0
pseudo_df['score'] = 1.0
pseudo_df['reactivity'] = pseudo_df['reactivity'].apply(lambda x: ast.literal_eval(x))
pseudo_df['deg_Mg_pH10'] = pseudo_df['deg_Mg_pH10'].apply(lambda x: ast.literal_eval(x))
pseudo_df['deg_pH10'] = pseudo_df['deg_pH10'].apply(lambda x: ast.literal_eval(x))
pseudo_df['deg_Mg_50C'] = pseudo_df['deg_Mg_50C'].apply(lambda x: ast.literal_eval(x))
pseudo_df['deg_50C'] = pseudo_df['deg_50C'].apply(lambda x: ast.literal_eval(x))

In [6]:
pseudo_st = pseudo_df[pseudo_df['seq_length'] == 107]
pseudo_lg = pseudo_df[pseudo_df['seq_length'] == 130]

pseudo_st['reactivity'] = pseudo_st['reactivity'].apply(lambda x: x[:68])
pseudo_st['deg_Mg_pH10'] = pseudo_st['deg_Mg_pH10'].apply(lambda x: x[:68])
pseudo_st['deg_pH10'] = pseudo_st['deg_pH10'].apply(lambda x: x[:68])
pseudo_st['deg_Mg_50C'] = pseudo_st['deg_Mg_50C'].apply(lambda x: x[:68])
pseudo_st['deg_50C'] = pseudo_st['deg_50C'].apply(lambda x: x[:68])

pseudo_lg['reactivity'] = pseudo_lg['reactivity'].apply(lambda x: x[:91])
pseudo_lg['deg_Mg_pH10'] = pseudo_lg['deg_Mg_pH10'].apply(lambda x: x[:91])
pseudo_lg['deg_pH10'] = pseudo_lg['deg_pH10'].apply(lambda x: x[:91])
pseudo_lg['deg_Mg_50C'] = pseudo_lg['deg_Mg_50C'].apply(lambda x: x[:91])
pseudo_lg['deg_50C'] = pseudo_lg['deg_50C'].apply(lambda x: x[:91])

/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [7]:
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
#base_train_data.head()

# add aug data

base_train_data = aug_data(base_train_data)
# base_train_data = base_train_data.append(pseudo_st)
base_train_data = base_train_data.reset_index(drop = True)
# base_train_data_lg = pseudo_lg.reset_index(drop = True)


device = torch.device('cuda')
BATCH_SIZE = 64

#base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
base_test_data = aug_data(base_test_data)

public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()

features, _ = preprocess(base_train_data, True)
features_tensor = torch.from_numpy(features)
dataset0 = VacDataset(features_tensor, base_train_data, None)

# features, _ = preprocess(base_train_data_lg, True)
# features_tensor = torch.from_numpy(features)
# dataset0_lg = VacDataset(features_tensor, base_train_data_lg, None)

features, _ = preprocess(public_df, True)
features_tensor = torch.from_numpy(features)
dataset1 = VacDataset(features_tensor, public_df, None)

features, _ = preprocess(private_df, True)
features_tensor = torch.from_numpy(features)
dataset2 = VacDataset(features_tensor, private_df, None)

loader0 = torch.utils.data.DataLoader(dataset0, BATCH_SIZE, shuffle=False, drop_last=False)
# loader0_lg = torch.utils.data.DataLoader(dataset0_lg, BATCH_SIZE, shuffle=False, drop_last=False)
loader1 = torch.utils.data.DataLoader(dataset1, BATCH_SIZE, shuffle=False, drop_last=False)
loader2 = torch.utils.data.DataLoader(dataset2, BATCH_SIZE, shuffle=False, drop_last=False)

public_df: (1258, 10)
private_df: (6010, 10)
(1, 107, 107, 3)
(1, 107, 107, 3)
(1, 130, 130, 3)


### Pretrain

In [8]:
def learn_from_batch_ae(model, data, device):
    seq = data["sequence"].clone()
    seq[:, :, :14] = F.dropout2d(seq[:, :, :14], p=0.3)
    target = data["sequence"][:, :, :14]
    out = model(seq.to(device), data["bpp"].to(device))
    loss = F.binary_cross_entropy(out, target.to(device))
    return loss


def train_ae(model, train_data, optimizer, lr_scheduler, epochs=10, device="cpu", 
             start_epoch: int = 0, start_it: int = 0, log_path: str = "./logs"): #10
    print(f"device: {device}")
    losses = []
    it = start_it
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    min_loss = 10.0
    min_loss_epoch = 0
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            optimizer.zero_grad()
            loss = learn_from_batch_ae(model, data, device)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            if lr_scheduler:
                lr_scheduler.step()
            loss_v = loss.item()
            losses.append(loss_v)
            it += 1
        loss_m = np.mean(losses)
        if loss_m < min_loss:
            min_loss_epoch = epoch
            min_loss = loss_m
        print(f'epoch: {epoch} loss: {loss_m}')
        losses = []
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    return dict(end_epoch=end_epoch, it=it, min_loss_epoch=min_loss_epoch)

In [9]:
import shutil

set_seed(123)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)

lr_scheduler = None
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AEModel()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
res = dict(end_epoch=0, it=0, min_loss_epoch=0)
epochs = [5, 5, 5, 5] # [5, 5, 5, 5]
for e in epochs:
    print('--loader0')
    res = train_ae(model, loader0, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
#     print('--loader0_lg')
#     res = train_ae(model, loader0_lg, optimizer, lr_scheduler, e, device=device,
#                    start_epoch=res["end_epoch"], start_it=res["it"])
    print('--loader1')
    res = train_ae(model, loader1, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    print('--loader2')
    res = train_ae(model, loader2, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])

epoch = res["min_loss_epoch"]
shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"model-{epoch}.pt"), "ae-model.pt")

--loader0
device: cuda


epoch: 0
epoch: 0 loss: 0.25064601679642995
epoch: 1
epoch: 1 loss: 0.1384060760339101
epoch: 2
epoch: 2 loss: 0.08038593033949534
epoch: 3
epoch: 3 loss: 0.03185466726620992
epoch: 4
epoch: 4 loss: 0.019618625019987426
--loader1
device: cuda


epoch: 5
epoch: 5 loss: 0.017772705294191837
epoch: 6
epoch: 6 loss: 0.01611646986566484
epoch: 7
epoch: 7 loss: 0.013704161671921612
epoch: 8
epoch: 8 loss: 0.011801821365952491
epoch: 9
epoch: 9 loss: 0.010440635774284602
--loader2
device: cuda


epoch: 10
epoch: 10 loss: 0.01808331135977456
epoch: 11
epoch: 11 loss: 0.014588844268879991
epoch: 12
epoch: 12 loss: 0.012971125731363576
epoch: 13
epoch: 13 loss: 0.012219857741543588
epoch: 14
epoch: 14 loss: 0.011251784831364738
--loader0
device: cuda


epoch: 15
epoch: 15 loss: 0.11141376870994767
epoch: 16
epoch: 16 loss: 0.14910804033279418
epoch: 17
epoch: 17 loss: 0.1133262633283933
epoch: 18
epoch: 18 loss: 0.08852919230858484
epoch: 19
epoch: 19 loss: 0.0644798181951046
--loader1
device: cuda


...

epoch: 40
epoch: 40 loss: 0.014415230730825917
epoch: 41
epoch: 41 loss: 0.013332009156967731
epoch: 42
epoch: 42 loss: 0.01705033711573862
epoch: 43
epoch: 43 loss: 0.030063067166570652
epoch: 44
epoch: 44 loss: 0.01620889058772554
--loader0
device: cuda


epoch: 45
epoch: 45 loss: 0.0104247345837454
epoch: 46
epoch: 46 loss: 0.010372398793697358
epoch: 47
epoch: 47 loss: 0.011601323901365201
epoch: 48
epoch: 48 loss: 0.030044919215142726
epoch: 49
epoch: 49 loss: 0.0157378122707208
--loader1
device: cuda


epoch: 50
epoch: 50 loss: 0.009795540105551481
epoch: 51
epoch: 51 loss: 0.009084979956969618
epoch: 52
epoch: 52 loss: 0.009306920599192381
epoch: 53
epoch: 53 loss: 0.011493802862241864
epoch: 54
epoch: 54 loss: 0.01057223421521485
--loader2
device: cuda


epoch: 55
epoch: 55 loss: 0.019030503562076927
epoch: 56
epoch: 56 loss: 0.016735949563456975
epoch: 57
epoch: 57 loss: 0.014369799279944693
epoch: 58
epoch: 58 loss: 0.013059445131728624
epoch: 59
epoch: 59 loss: 0.012738071332507312


'ae-model.pt'

### Training

In [10]:
def MCRMSE(y_true, y_pred):
    colwise_mse = torch.mean(torch.square(y_true - y_pred), dim=1)
    return torch.mean(torch.sqrt(colwise_mse), dim=1)


def sn_mcrmse_loss(predict, target, signal_to_noise):
    loss = MCRMSE(target, predict)
    weight = 0.5 * torch.log(signal_to_noise + 1.01)
    loss = (loss * weight).mean()
    return loss


def learn_from_batch(model, data, optimizer, lr_scheduler, device):
    optimizer.zero_grad()

    out = model(data["sequence"].to(device), data["bpp"].to(device))
    out = out[:, :data["label"].shape[1]]
    signal_to_noise = data["signal_to_noise"] * data["score"]
    loss = sn_mcrmse_loss(out, data["label"].to(device), signal_to_noise.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    if lr_scheduler:
        lr_scheduler.step()
    return out, loss

def predict_batch(model, data, device):
    # batch x seq_len x target_size
    with torch.no_grad():
        pred = model(data["sequence"].to(device), data["bpp"].to(device))
        pred = pred.detach().cpu().numpy()
    return_values = []
    ids = data["ids"]
    for idx, p in enumerate(pred):
        id_ = ids[idx]
        assert p.shape == (model.pred_len, len(target_cols))
        for seqpos, val in enumerate(p):
            assert len(val) == len(target_cols)
            dic = {key: val for key, val in zip(target_cols, val)}
            dic["id_seqpos"] = f"{id_}_{seqpos}"
            return_values.append(dic)
    return return_values


def predict_data(model, loader, device, batch_size):
    model.eval()
    data_list = []
    for i, data in enumerate(progress_bar(loader)):
        data_list += predict_batch(model, data, device)
    expected_length = model.pred_len * len(loader) * batch_size
#     assert len(data_list) == expected_length, f"len = {len(data_list)} expected = {expected_length}"
    return data_list

def evaluate(model, valid_data, device):
    model.eval()
    loss_list = []
    mcrmse = []
    for i, data in enumerate(valid_data):
        with torch.no_grad():
            y = model(data["sequence"].to(device), data["bpp"].to(device))
            y = y[:, :68]
            mcrmse_ = MCRMSE(data["label"].to(device), y)[data["signal_to_noise"] > 1]
            mcrmse.append(mcrmse_.mean().item())
            loss = sn_mcrmse_loss(y, data["label"].to(device), data["signal_to_noise"].to(device))
            loss_list.append(loss.item())
    model.train()
    return dict(loss=np.mean(loss_list), mcmse=np.mean(mcrmse))

def train(model, train_data, valid_data, train_data_lg, valid_data_lg, optimizer, lr_scheduler, epochs=10, device="cpu",
          start_epoch: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    writer = SummaryWriter(log_path)
    it = 0
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    min_eval_loss = 10.0
    min_eval_epoch = None
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        
        for i, (data, data_lg) in enumerate(itertools.zip_longest(train_data, train_data_lg)):
            if data_lg is not None:
                _, loss_lg = learn_from_batch(model, data_lg, optimizer, lr_scheduler, device)
                losses.append(loss_lg.item())
                
            _, loss = learn_from_batch(model, data, optimizer, lr_scheduler, device)
            loss_v = loss.item()
            writer.add_scalar('loss', loss_v, it)
            losses.append(loss_v)
            it += 1
        print(f'epoch: {epoch} loss: {np.mean(losses)}')
        losses = []

        eval_result = evaluate(model, valid_data, device)
        eval_loss = eval_result["loss"]
        if eval_loss <= min_eval_loss:
            min_eval_epoch = epoch
            min_eval_loss = eval_loss
            min_mcrmse = eval_result["mcmse"]
            valid_pred = predict_data(model, valid_data, device, BATCH_SIZE)

        print(f"eval loss: {eval_loss} {eval_result['mcmse']}")
        writer.add_scalar(f"evaluate/loss", eval_loss, epoch)
        writer.add_scalar(f"evaluate/mcmse", eval_result["mcmse"], epoch)
        model.train()
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
        
    print(f'min eval loss: {min_eval_loss} min mcrmse: {min_mcrmse} epoch {min_eval_epoch}')
    return min_eval_epoch, valid_pred

In [11]:
device = torch.device('cuda')
BATCH_SIZE = 64

# categorical value for target (used for stratified kfold)
#def add_y_cat(df):
#    target_mean = df['reactivity'].apply(np.mean) + \
#                  df['deg_Mg_pH10'].apply(np.mean) + \
#                  df['deg_Mg_50C'].apply(np.mean) 
                  #df['deg_pH10'].apply(np.mean) + \
                  #df['deg_50C'].apply(np.mean)
#    df['y_cat'] = pd.qcut(np.array(target_mean), q=20).codes
    
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)

base_train_data = aug_data(base_train_data)
base_train_data = base_train_data.append(pseudo_st)
base_train_data = base_train_data.reset_index(drop = True)
base_train_data_lg = pseudo_lg.reset_index(drop = True)
#add_y_cat(base_train_data)

samples = base_train_data
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
split = ShuffleSplit(n_splits=5, test_size=.1)
# kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 123)
kf = GroupKFold(n_splits=5)
    
ids = samples.reset_index()["id"]

set_seed(123)

oof_list = []
for fold, ((train_index, test_index), (train_index_lg, test_index_lg)) in enumerate(zip(kf.split(samples, groups=ids), 
                                                     kf.split(base_train_data_lg, groups=base_train_data_lg['id']))): #split.split(samples) / kf
    print(f"fold: {fold}")
    train_df = samples.loc[train_index].reset_index()
    val_df = samples.loc[test_index].reset_index()
    train_loader = create_loader(train_df, BATCH_SIZE)
    valid_loader = create_loader(val_df, BATCH_SIZE)
    print(train_df.shape, val_df.shape)
    
    train_df_lg = base_train_data_lg.loc[train_index_lg].reset_index()
    val_df_lg = base_train_data_lg.loc[test_index_lg].reset_index()
    train_loader_lg = create_loader(train_df_lg, BATCH_SIZE)
    valid_loader_lg = create_loader(val_df_lg, BATCH_SIZE)
    print(train_df_lg.shape, val_df_lg.shape)
    
    ae_model = AEModel()
    state_dict = torch.load("./ae-model.pt")
    ae_model.load_state_dict(state_dict)
    del state_dict
    
    model = FromAeModel(ae_model.seq, pred_len=91)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    lr_scheduler = None
    
    epoch, valid_pred = train(model, train_loader, valid_loader, train_loader_lg, valid_loader_lg, optimizer, lr_scheduler, 120, device=device,
                  log_path=f"logs/{fold}") #150 #200 epochs
    oof_list += valid_pred
    shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"./model-{epoch}.pt"), f"model_prediction/model-{fold}.pt")
    del model

fold: 0
(1, 107, 107, 3)
(1, 107, 107, 3)
(4343, 23) (1086, 23)
(1, 130, 130, 3)
(1, 130, 130, 3)
(2404, 16) (601, 16)
device: cuda


epoch: 0
epoch: 0 loss: 0.18869469190160354


eval loss: 0.21509586582567805 0.30248562270230833
epoch: 1
epoch: 1 loss: 0.15431082517049252


eval loss: 0.19450895108046576 0.2751677992319841
epoch: 2
epoch: 2 loss: 0.1409647957733456


eval loss: 0.18319475955616493 0.25941055190707996
epoch: 3
epoch: 3 loss: 0.13313073580271603


...

eval loss: 0.1513173512493379 0.2144563306910257
epoch: 44
epoch: 44 loss: 0.08316457605766085


eval loss: 0.1502836576580044 0.21343520745071554
epoch: 45
epoch: 45 loss: 0.08250454278926725
eval loss: 0.15168532852296476 0.21461554174541164
epoch: 46
epoch: 46 loss: 0.08330500412101716


eval loss: 0.14998232652076848 0.21231635149881345
epoch: 47
epoch: 47 loss: 0.08148964118340139
eval loss: 0.15085586492954295 0.2139350229036206
epoch: 48
epoch: 48 loss: 0.08550705491517908
eval loss: 0.15149598845236836 0.21486822717081336
epoch: 49
epoch: 49 loss: 0.08024550149466195


eval loss: 0.14994062148684467 0.2125610639671116
epoch: 50
epoch: 50 loss: 0.08232475354676938


eval loss: 0.14898649745066386 0.21158178663776644
epoch: 51
epoch: 51 loss: 0.08449402850395704
eval loss: 0.15415241489753018 0.21834847910724364
epoch: 52
epoch: 52 loss: 0.09454252345014136
eval loss: 0.15384900703917445 0.2176564978719071
epoch: 53
epoch: 53 loss: 0.09798771750010828
eval loss: 0.17182953791857777 0.24088143876552007
epoch: 54
epoch: 54 loss: 0.1798150153972822
eval loss: 0.30218118847670394 0.41950147150068423
epoch: 55
epoch: 55 loss: 0.19540211495562806
eval loss: 0.2549729346478726 0.35476433209609287
epoch: 56
epoch: 56 loss: 0.14304057193366676
eval loss: 0.19814231572960628 0.27862812504768675
epoch: 57
epoch: 57 loss: 0.11070869790839813
eval loss: 0.1550825360079942 0.2200011036097438
epoch: 58
epoch: 58 loss: 0.09012958632048676
eval loss: 0.152252462898259 0.21588831349996976
epoch: 59
epoch: 59 loss: 0.09807063841169283
eval loss: 0.1575332492313526 0.22288476838261628
epoch: 60
epoch: 60 loss: 0.09680505777637356
eval loss: 0.1546795034212294 0.219317

In [12]:
oof_df = pd.DataFrame(oof_list, columns=["id_seqpos"] + target_cols)
oof_df.to_csv('validation_aepytorch.csv', index=False)

### Prediction

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
base_test_data = aug_data(base_test_data)

public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()
pub_loader = create_loader(public_df, BATCH_SIZE, is_test=True)
pri_loader = create_loader(private_df, BATCH_SIZE, is_test=True)
pred_df_list = []
c = 0
for fold in range(5):
    model_load_path = f"./model_prediction/model-{fold}.pt"
    ae_model0 = AEModel()
    ae_model1 = AEModel()
    model_pub = FromAeModel(pred_len=107, seq=ae_model0.seq)
    model_pub = model_pub.to(device)
    model_pri = FromAeModel(pred_len=130, seq=ae_model1.seq)
    model_pri = model_pri.to(device)
    state_dict = torch.load(model_load_path, map_location=device)
    model_pub.load_state_dict(state_dict)
    model_pri.load_state_dict(state_dict)
    del state_dict

    data_list = []
    data_list += predict_data(model_pub, pub_loader, device, BATCH_SIZE)
    data_list += predict_data(model_pri, pri_loader, device, BATCH_SIZE)
    pred_df = pd.DataFrame(data_list, columns=["id_seqpos"] + target_cols)
    #print(pred_df.head())
    #print(pred_df.tail())
    pred_df_list.append(pred_df)
    c += 1
data_dic = dict(id_seqpos=pred_df_list[0]["id_seqpos"])
for col in target_cols:
    vals = np.zeros(pred_df_list[0][col].shape[0])
    for df in pred_df_list:
        vals += df[col].values
    data_dic[col] = vals / float(c)
pred_df_avg = pd.DataFrame(data_dic, columns=["id_seqpos"] + target_cols)
#print(pred_df_avg.head())
#pred_df_avg.to_csv("./submission_all.csv", index=False)

##### submission final
sample_df = pd.read_csv('../OpenVaccine/sample_submission.csv')
target_cols = [c for c in sample_df.columns if c != 'id_seqpos']
list_id = list(sample_df.id_seqpos.values)
output = {}
output_df = pd.DataFrame({'id_seqpos': sample_df.id_seqpos.values})

for c in target_cols:
    output_values = []
    x = pred_df_avg.groupby('id_seqpos')[c].mean().reset_index()
    print(x.shape)
    output_df = pd.merge(output_df, x, on='id_seqpos')
    
output_df.to_csv('submission_aepytorch.csv', index=False)
print(output_df.shape)

public_df: (1258, 10)
private_df: (6010, 10)
(1, 107, 107, 3)
(1, 130, 130, 3)


(457953, 2)
(457953, 2)
(457953, 2)
(457953, 2)
(457953, 2)
(457953, 6)


In [14]:
output_df.head(10)

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.647781,0.634589,1.880130,0.497652,0.708136
1,id_00073f8be_1,1.988751,2.927485,3.775051,2.922500,2.571540
2,id_00073f8be_2,1.441579,0.613249,0.685714,0.713150,0.676650
3,id_00073f8be_3,1.192090,1.029909,1.095550,1.474018,1.638364
4,id_00073f8be_4,0.809244,0.525973,0.460842,0.805732,0.854745
5,id_00073f8be_5,0.705192,0.513918,0.534220,0.657263,0.705889
6,id_00073f8be_6,0.739668,0.949133,0.950112,1.003862,0.979089
7,id_00073f8be_7,0.921899,0.937763,1.101304,0.940724,1.346590
8,id_00073f8be_8,0.196084,0.906140,0.817736,0.971482,0.708860
9,id_00073f8be_9,0.052724,0.208538,0.265326,0.280487,0.396252
